In [ ]:
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime
import time
import warnings ; warnings.warn = lambda *args,**kwargs: None
from tabulate import tabulate


In [19]:
def timestamp():
    return datetime.now().strftime("[%m/%d/%y - %H:%M] ")
def get_date():
    return datetime.now().strftime("%m/%d/%y")
def get_time():
    return datetime.now().strftime("%H:%M")
def get_date_time():
    return datetime.now().strftime("%m/%d/%y %H:%M")
def get_time_difference(entry1,entry2,verbose=False,unit='minutes'):
    t1_string = entry1['date time']
    t1 = datetime.strptime(t1_string, "%m/%d/%y %H:%M")

    t2_string = entry2['date time']
    t2 = datetime.strptime(t2_string, "%m/%d/%y %H:%M")
    if unit == 'minutes':
        factor=60
    elif unit == 'seconds':
        factor=1
    difference_minutes = int((t2-t1).seconds/factor)
    if verbose:
        print("")
        print(f"start: ".rjust(16)+f"{t1}")
        print(f"finish: ".rjust(16)+f"{t2}")
        print(f"difference: ".rjust(16)+f"{difference_minutes:.0f} {unit}")
        
    return difference_minutes
def get_date_specify_time(timestamp):
    return f'{get_date()} {timestamp}'
example_events = [
    {
        'date time': f'{get_date()} 01:18',
        'event':'START RECORD', 
        'classification':'start record'
    }
]

class events_analyzer():
    def __init__(self, records = None, verbose = False):
        self.verbose = verbose
        self.print(f"\n{timestamp()} INITIALIZING EVENTS ANALYZER")
        self.events = pd.DataFrame(columns=['date time','order','event','classification','quantity'])

        if not pd.isnull(records).any():
            for entry in records:
                self.append_entry(entry)

        else:
            entry = {
                'event':'START RECORD',
                'classification':'start record'
            }
            self.append_entry(entry)
    def print(self,text,end='\n'):
        if self.verbose:
            print(text,end=end)
    
    def print_entry(self,entry:dict):
        max_key_size = max([len(key) for key in self.events.columns])
        self.print("")
        for key, value in entry.items():
            self.print(key.rjust(max_key_size)+": "+value)    
    
    def append_entry(self,entry):
        if 'date time' not in entry.keys():
            entry['date time'] = get_date_time()
        self.print(f"\n{timestamp()} APPEND NEW ENTRY:")
        self.print_entry(entry)
        self.events.loc[self.events.index.size] = entry
        self.order()

    def wake_up(self, date_time = get_date_time()):
        entry = {}
        entry['date time'] = date_time
        self.print(f"\n{timestamp()} RECORDING WAKE UP",end="")
        entry['event'] = 'wake up'
        entry['classification'] = 'end sleep'

        starts = ['START RECORD','go to sleep']
        nstarts = len(set(starts).intersection(set(self.events['event'])))

        if nstarts>0:
            start_entry = dict(self.events[self.events['event'].isin(starts)].iloc[-1])
            difference = get_time_difference(start_entry,entry)
            self.print(f" (Sleep time = {difference} minutes)")
            get_time_difference(start_entry,entry,verbose=self.verbose)
            entry['quantity'] = f"{difference} minutes"
        else:
            self.print("")
        self.append_entry(entry)
        
    def go_to_sleep(self, date_time = get_date_time()):
        entry = {}
        entry['date time'] = date_time
        self.print(f"\n{timestamp()} RECORDING GO TO SLEEP")
        entry['event'] = 'go to sleep'
        entry['classification'] = 'start sleep'
        self.append_entry(entry)
        
    def display(self):
        self.print(f"\n{timestamp()} DISPLAY EVENT RECORD:\n")
        table = tabulate(self.events.fillna(''), headers = 'keys', tablefmt = 'fancy_grid', showindex=False)
        self.print(table)

    def supplement(self, classification, date_time = get_date_time(), quantity=None):
        entry = {}
        entry['date time'] = date_time
        entry['event'] = 'supplement'
        entry['classification'] = classification
        self.print(f'\n{timestamp()} RECORDING SUPPLEMENT: "{classification}"',end='')
        if not pd.isnull(quantity):
            entry['quantity'] = str(quantity)
            self.print(f', quantity = "{quantity}"')
        else:
            self.print("")
        self.append_entry(entry)

    def diaper(self, date_time = get_date_time(), classification=None, quantity=None):
        entry = {}
        entry['date time'] = date_time
        entry['event'] = 'diaper change'
        
        self.print(f'\n{timestamp()} RECORDING DIAPER CHANGE',end='')

        if not pd.isnull(classification):
            entry['classification'] = classification
            self.print(f': "{classification}"',end='')

        if not pd.isnull(quantity):
            entry['quantity'] = str(quantity)
            self.print(f', quantity = "{quantity}"',end='')

        else:
            self.print("")
        self.append_entry(entry)

    def feed(self, date_time = get_date_time(), classification=None, quantity=None):
        entry = {}
        entry['date time'] = date_time
        entry['event'] = 'feed'
        
        self.print(f'\n{timestamp()} RECORDING FEED',end='')

        if not pd.isnull(classification):
            entry['classification'] = classification
            self.print(f': "{classification}"',end='')

        if not pd.isnull(quantity):
            entry['quantity'] = str(quantity)
            self.print(f', quantity = "{quantity}"',end='')
            
        else:
            self.print("")
        self.append_entry(entry)

    def mood(self, classification, date_time = get_date_time()):
        entry = {}
        entry['date time'] = date_time
        entry['event'] = 'mood'
        entry['classification'] = classification
        self.print(f'\n{timestamp()} RECORDING MOOD: "{classification}')
        self.append_entry(entry)

    def order(self):
        unordered_index = self.events[self.events['order'].isnull()].index
        unordered_timestamps = self.events['date time'][unordered_index].unique()
        for timestamp in unordered_timestamps:
            mask = self.events['date time']==timestamp
            self.events['order'][mask] = range(1,sum(mask)+1)
        self.events['order'] = self.events['order'].astype(int)
        
test = events_analyzer(records=example_events,verbose=True)
test.wake_up(date_time = f'{get_date()} 01:30')
test.supplement("gas drops",quantity="0.3 mL", date_time = f'{get_date()} 01:30')
test.diaper(date_time = f'{get_date()} 01:30')
test.feed(date_time = f'{get_date()} 01:35')
test.mood("normal", date_time = f'{get_date()} 01:50')
test.go_to_sleep(date_time = f'{get_date()} 02:00')
test.wake_up(date_time = f'{get_date()} 06:30')
test.display()


[04/21/24 - 00:42]  INITIALIZING EVENTS ANALYZER

[04/21/24 - 00:42]  APPEND NEW ENTRY:

     date time: 04/21/24 01:18
         event: START RECORD
classification: start record

[04/21/24 - 00:42]  RECORDING WAKE UP (Sleep time = 12 minutes)

         start: 2024-04-21 01:18:00
        finish: 2024-04-21 01:30:00
    difference: 12 minutes

[04/21/24 - 00:42]  APPEND NEW ENTRY:

     date time: 04/21/24 01:30
         event: wake up
classification: end sleep
      quantity: 12 minutes

[04/21/24 - 00:42]  RECORDING SUPPLEMENT: "gas drops", quantity = "0.3 mL"

[04/21/24 - 00:42]  APPEND NEW ENTRY:

     date time: 04/21/24 01:30
         event: supplement
classification: gas drops
      quantity: 0.3 mL

[04/21/24 - 00:42]  RECORDING DIAPER CHANGE

[04/21/24 - 00:42]  APPEND NEW ENTRY:

     date time: 04/21/24 01:30
         event: diaper change

[04/21/24 - 00:42]  RECORDING FEED

[04/21/24 - 00:42]  APPEND NEW ENTRY:

     date time: 04/21/24 01:35
         event: feed

[04/21/24 

In [ ]:
test = events_analyzer(records=example_events,verbose=True)

for day in range(1,8):
    
# test.wake_up(date_time = f'{get_date()} 01:30')
# test.supplement("gas drops",quantity="0.3 mL", date_time = f'{get_date()} 01:30')
# test.diaper(date_time = f'{get_date()} 01:30')
# test.feed(date_time = f'{get_date()} 01:35')
# test.mood("normal", date_time = f'{get_date()} 01:50')
# test.go_to_sleep(date_time = f'{get_date()} 02:00')
# test.wake_up(date_time = f'{get_date()} 06:30')
# test.display()

In [18]:
# creating a DataFrame
df = test.events.copy().fillna('')
 
formats = [
    
    "plain",
    "simple",
    "github",
    "grid",
    "fancy_grid",
    "pipe",
    "orgtbl",
    "jira",
    "presto",
    "pretty",
    "psql",
    "rst",
    "mediawiki",
    "moinmoin",
    "youtrack",
    "html",
    "latex",
    "latex_raw",
    "latex_booktabs",
    "textile"

]
# displaying the DataFrame

for fmt in formats:
    print(f"\nFORMAT = '{fmt}'\n")
    print(tabulate(df, headers = 'keys', tablefmt = fmt, showindex=False))
    print("\n")


FORMAT = 'plain'

date time         order  event          classification    quantity
04/21/24 01:18        1  START RECORD   start record
04/21/24 01:30        1  wake up        end sleep         12 minutes
04/21/24 01:30        2  supplement     gas drops         0.3 mL
04/21/24 01:30        3  diaper change
04/21/24 01:35        1  feed
04/21/24 01:50        1  mood           normal
04/21/24 02:00        1  go to sleep    start sleep
04/21/24 06:30        1  wake up        end sleep         270 minutes



FORMAT = 'simple'

date time         order  event          classification    quantity
--------------  -------  -------------  ----------------  -----------
04/21/24 01:18        1  START RECORD   start record
04/21/24 01:30        1  wake up        end sleep         12 minutes
04/21/24 01:30        2  supplement     gas drops         0.3 mL
04/21/24 01:30        3  diaper change
04/21/24 01:35        1  feed
04/21/24 01:50        1  mood           normal
04/21/24 02:00        1  go